In [ ]:
import torch

In [ ]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [ ]:
!git clone https://github.com/LinDuoming/LogEval.git

Cloning into 'LogEval'...
remote: Enumerating objects: 304, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 304 (delta 21), reused 29 (delta 11), pack-reused 254 (from 1)
Receiving objects: 100% (304/304), 3.57 MiB | 13.80 MiB/s, done.
Resolving deltas: 100% (96/96), done.


In [ ]:
%cd LogEval

/content/LogEval


In [ ]:
%cd LogEval
!pip install -r requirements.txt

!pip install transformers accelerate sentencepiece
!pip install -U bitsandbytes

[Errno 2] No such file or directory: 'LogEval'
/content/LogEval
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 5.7 MB/s eta 0:00:00
ERROR: Ignored the following versions that require a different python version: 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11
ERROR: Could not find a version that satisfies the requirement pybabel==0.0.0.dev0 (from versions: none)
ERROR: No matching distribution found for pybabel==0.0.0.dev0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!ls -l
!ls -l app.py datasrc.py

total 140
-rw-r--r-- 1 root root  7128 Jul 11 13:41 app.py
-rw-r--r-- 1 root root   100 Jul 11 13:41 babel.cfg
-rw-r--r-- 1 root root   498 Jul 11 13:41 datasrc.py
-rw-r--r-- 1 root root   105 Jul 11 13:41 index.html
-rw-r--r-- 1 root root 74412 Jul 11 13:41 leaderboard.csv
-rw-r--r-- 1 root root  6083 Jul 11 13:41 messages.pot
-rw-r--r-- 1 root root   209 Jul 11 13:41 news.json
drwxr-xr-x 3 root root  4096 Jul 11 13:41 old_proj
-rw-r--r-- 1 root root  5838 Jul 11 13:41 readme.md
-rw-r--r-- 1 root root   335 Jul 11 13:41 requirements.txt
drwxr-xr-x 2 root root  4096 Jul 11 13:41 static
-rw-r--r-- 1 root root    25 Jul 11 13:41 submitted.csv
drwxr-xr-x 2 root root  4096 Jul 11 13:41 templates
drwxr-xr-x 3 root root  4096 Jul 11 13:41 translations
-rw-r--r-- 1 root root 7128 Jul 11 13:41 app.py
-rw-r--r-- 1 root root  498 Jul 11 13:41 datasrc.py


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from huggingface_hub import login

hf_token = "your_token"
login(token=hf_token)

model_id_mistral = "mistralai/Mistral-7B-Instruct-v0.2"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

print(f"Loading tokenizer for {model_id_mistral}...")
tokenizer_mistral = AutoTokenizer.from_pretrained(model_id_mistral)
print("Tokenizer loaded.")

print(f"Loading model {model_id_mistral} with 4-bit quantization...")
model_mistral = AutoModelForCausalLM.from_pretrained(
    model_id_mistral,
    quantization_config=bnb_config,
    device_map="auto"
)
print("Mistral-7B-Instruct-v0.2 loaded successfully!")

messages = [
    {"role": "user", "content": "What is log parsing?"}
]
input_ids = tokenizer_mistral.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")

print("\nGenerating a test response from Mistral-7B-Instruct-v0.2...")
outputs = model_mistral.generate(input_ids, max_new_tokens=200, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
response_mistral = tokenizer_mistral.decode(outputs[0], skip_special_tokens=True)
print("Mistral Response:")
print(response_mistral)

Loading tokenizer for mistralai/Mistral-7B-Instruct-v0.2...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Mistral-7B-Instruct-v0.2 loaded successfully!

Generating a test response from Mistral-7B-Instruct-v0.2...
Mistral Response:
[INST] What is log parsing? [/INST] Log parsing is the process of extracting and analyzing data from log files, which are records of events that have occurred on a system or application. Log files can contain a wide range of data, including timestamps, user activity, system events, error messages, and other information.

The purpose of log parsing is to make sense of this data and extract meaningful insights from it. Log parsing can be used to monitor system performance, troubleshoot issues, detect security threats, and gain a better understanding of user behavior.

Log parsing can be performed manually, using text editors or search tools to search for specific patterns or keywords in log files. However, large or complex log files often require the use of specialized log parsing tools or scripts to automate the process and extract more sophisticated insights. The

In [ ]:

def get_mistral_response(log_entry, instruction, model, tokenizer, max_new_tokens=200):

    messages = [
        {"role": "system", "content": "You are a helpful assistant for log analysis. Provide concise and accurate answers."},
        {"role": "user", "content": f"{instruction}\nLog entry: {log_entry}"}
    ]


    input_ids = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to(model.device)


    outputs = model.generate(input_ids,
                             max_new_tokens=max_new_tokens,
                             do_sample=True,
                             temperature=0.7,
                             top_k=50,
                             top_p=0.95,
                             pad_token_id=tokenizer.eos_token_id) # اضافه کردن pad_token_id برای جلوگیری از هشدار


    response = tokenizer.decode(outputs[0], skip_special_tokens=True)


    if "[/INST]" in response:
        response = response.split("[/INST]", 1)[1].strip()

    return response

In [ ]:

log_example_parse = "12-10-2023 10:30:00 INFO User 'admin' logged in from IP 192.168.1.100."
instruction_parse = "Parse the following log entry into a template format, replacing variable parts with a wildcard <*>:"

response_parse = get_mistral_response(log_example_parse, instruction_parse, model_mistral, tokenizer_mistral)
print(f"\nResponse for Log Parsing:\n{response_parse}")


log_example_anomaly = "12-10-2023 11:45:10 ERROR High CPU utilization detected on serverXYZ (98%)."
instruction_anomaly = "Review the log entry and explicitly mark it as 'normal' or 'abnormal', only output 'normal' or 'abnormal'."

response_anomaly = get_mistral_response(log_example_anomaly, instruction_anomaly, model_mistral, tokenizer_mistral, max_new_tokens=50) # max_new_tokens کمتر برای پاسخ‌های کوتاه
print(f"\nResponse for Log Anomaly Detection:\n{response_anomaly}")


Response for Log Parsing:
Template format: [DD-MM-YYYY] [HH:MM:SS] [Level] User <*> logged in from IP <*>

So the parsed log entry would look like: [12-10-2023] [10:30:00] [INFO] User 'admin' logged in from IP 192.168.1.100.

Response for Log Anomaly Detection:
abnormal.


In [ ]:
!pip install rouge_score scikit-learn Levenshtein

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 63.3 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=98517c070c3b1de2755ee75f8990c731d653d50dfc3f52217b4a1206d437fc0c
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
%cd LogEval

!cp static/dataset.zip .


!unzip -o dataset.zip

[Errno 2] No such file or directory: 'LogEval'
/content/LogEval
Archive:  dataset.zip
   creating: dataset/
   creating: dataset/log_anormaly/
  inflating: dataset/log_anormaly/evaluation_set_chinese_abnormal_detect_few_shot.json  
  inflating: dataset/log_anormaly/evaluation_set_chinese_abnormal_detect_zero_shot.json  
  inflating: dataset/log_anormaly/evaluation_set_english_abnormal_detect_few_shot.json  
  inflating: dataset/log_anormaly/evaluation_set_english_abnormal_detect_zero_shot.json  
   creating: dataset/log_diagnosis/
  inflating: dataset/log_diagnosis/fault_diagnosis_few_shot_evaluation_set_chinese.json  
  inflating: dataset/log_diagnosis/fault_diagnosis_few_shot_evaluation_set_english.json  
  inflating: dataset/log_diagnosis/fault_diagnosis_zero_shot_evaluation_set_chinese.json  
  inflating: dataset/log_diagnosis/fault_diagnosis_zero_shot_evaluation_set_english.json  
   creating: dataset/log_parsing/
  inflating: dataset/log_parsing/evaluation_set_chinese_log_parsing

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from huggingface_hub import login
import os
import pandas as pd
import json
import datetime
import re
import time
from sklearn.metrics import f1_score, accuracy_score
from rouge_score import rouge_scorer
import Levenshtein
from sentence_transformers import SentenceTransformer, util

GENERAL_SUMMARIZATION_PROMPTS_EN = {
    "Prompt1_ConciseEvent": "Summarize the following log entries concisely. Focus on identifying the main events and their outcomes. Provide a brief overview.\nLogs:\n<INSERT_LOGS>\nSummary:",
    "Prompt2_SystemAdminRole": "You are a diligent System Administrator monitoring system health. Summarize the following log entries. Focus on critical system events, errors, warnings, and any unusual activity that might affect system stability or performance. What are the key takeaways for system health?\nLogs:\n<INSERT_LOGS>\nSystem Status Summary:",
    "Prompt3_PatternDetection": "Analyze the following log entries for recurring patterns, repeated errors, or frequent events. Summarize these repetitions and their potential implications for the system's operation.\nLogs:\n<INSERT_LOGS>\nRecurring Events Summary:",
    "Prompt4_ProblemAnalysis": "Assume there's an ongoing issue. Summarize the following logs to pinpoint the root cause or primary problem affecting the system. Explain the findings clearly.\nLogs:\n<INSERT_LOGS>\nProblem Analysis Summary:"
}


sbert_model = None
try:
    print("Loading SentenceTransformer model for Cosine Similarity...")
    sbert_model = SentenceTransformer('all-MiniLM-L6-v2', device='cuda' if torch.cuda.is_available() else 'cpu')
    print("SentenceTransformer model loaded.")
except Exception as e:
    print(f"Warning: Could not load SentenceTransformer model for Cosine Similarity: {e}")
    print("Cosine Similarity will not be calculated. Please ensure 'sentence_transformers' is installed and check available RAM.")
    sbert_model = None

hf_token = os.environ.get("HF_TOKEN", "your_token")

try:
    login(token=hf_token)
    print("Logged in to Hugging Face successfully.")
except Exception as e:
    print(f"Failed to log in to Hugging Face: {e}")
    print("Please ensure your HF_TOKEN is correct and has access to the model.")

model_id_mistral = "mistralai/Mistral-7B-Instruct-v0.2"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

try:
    print(f"Loading tokenizer for {model_id_mistral}...")
    tokenizer_mistral = AutoTokenizer.from_pretrained(model_id_mistral)
    print("Tokenizer loaded.")

    print(f"Loading model {model_id_mistral} with 4-bit quantization and CPU offload...")
    model_mistral = AutoModelForCausalLM.from_pretrained(
        model_id_mistral,
        quantization_config=bnb_config,
        device_map="auto",

    )
    print("Mistral-7B-Instruct-v0.2 loaded successfully with offload!")
except Exception as e:
    print(f"Error loading Mistral model: {e}")
    model_mistral = None
    tokenizer_mistral = None
    print("Mistral-7B will not be available for benchmarking.")


def get_mistral_response(log_entry, instruction_template, model, tokenizer, max_new_tokens=200):
    if model is None or tokenizer is None:
        return "Error in generating response.", 0.0, 0, 0, 0

    if "<INSERT_LOGS>" in instruction_template:
        final_instruction = instruction_template.replace("<INSERT_LOGS>", log_entry)
    else:
        final_instruction = f"{instruction_template}\nLog entry: {log_entry}"

    messages = [
        {"role": "system", "content": "You are a helpful assistant for log analysis. Provide concise and accurate answers."},
        {"role": "user", "content": final_instruction}
    ]

    try:
        start_time = time.time()
        input_ids = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to(model.device)
        outputs = model.generate(input_ids,
                                 max_new_tokens=max_new_tokens,
                                 do_sample=True,
                                 temperature=0.7,
                                 top_k=50,
                                 top_p=0.95,
                                 pad_token_id=tokenizer.eos_token_id)
        end_time = time.time()

        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        generated_tokens_count = len(outputs[0]) - len(input_ids[0])

        if "[/INST]" in response:
            response = response.split("[/INST]", 1)[1].strip()


        security_keywords_in_summary = 0
        security_keywords_in_log = 0


        return response, (end_time - start_time), generated_tokens_count, security_keywords_in_summary, security_keywords_in_log

    except Exception as e:
        print(f"Error generating response: {e}")
        return "Error in generating response.", 0.0, 0, 0, 0

def _get_dataset(task_type, lang, instr):

    task_file_name_map = { "summary": "log_summary" }
    lang_map = {"English": "english"}
    instr_map = {"zero-shot": "zero_shot"}
    task_subfolder_map = { "summary": "log_summary" }

    if task_type not in task_file_name_map or lang not in lang_map or instr not in instr_map:
        print(f"Error: Invalid task_type ({task_type}), lang ({lang}) or instr ({instr}) provided.")
        return pd.DataFrame(), {}

    if task_type not in task_subfolder_map:
        print(f"Error: Task subfolder mapping not found for {task_type}. Skipping.")
        return pd.DataFrame(), {}


    file_name = f"{task_file_name_map[task_type]}_{instr_map[instr]}_evaluation_set_{lang_map[lang]}.json"

    file_path = os.path.join("dataset", task_subfolder_map[task_type], file_name)

    print(f"Attempting to load data from: {file_path}")
    print(f"Check existence for this path: {os.path.exists(file_path)}")

    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        questions_df = pd.DataFrame(data)

        labels_dict = {row["id"]: row["output"] for _, row in questions_df.iterrows()}
        questions_df['log_entry'] = questions_df['input']

        print(f"Successfully loaded {len(questions_df)} samples for {task_type}, {lang}, {instr}.")
        return questions_df, labels_dict

    except FileNotFoundError:
        print(f"Warning: Data file not found for {task_type} ({lang}, {instr}) at {file_path}. Skipping.")
        return pd.DataFrame(), {}
    except json.JSONDecodeError:
        print(f"Error: Could not decode JSON from {file_path}. Check file format. Skipping.")
        return pd.DataFrame(), {}
    except KeyError as e:
        print(f"Error: Missing expected key in JSON data: {e}. Check data structure. Skipping.")
        return pd.DataFrame(), {}
    except Exception as e:
        print(f"An unexpected error occurred while loading dataset for {task_type} ({lang}, {instr}): {e}. Skipping.")
        return pd.DataFrame(), {}



def calculate_summary_metrics(predict_raw, label_raw, log_entry_raw=None):
    predict = predict_raw.strip()
    label = label_raw.strip()

    if not predict and not label: return 1.0, 1.0, 1.0, 0, 0
    if not predict or not label: return 0.0, 0.0, 0.0, 0, 0

    scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)
    scores = scorer.score(label, predict)
    rouge1_f1 = scores['rouge1'].fmeasure

    cosine_sim_score = 0.0
    if sbert_model is not None:
        try:
            embeddings1 = sbert_model.encode(predict, convert_to_tensor=True)
            embeddings2 = sbert_model.encode(label, convert_to_tensor=True)
            cosine_sim_score = float(util.cos_sim(embeddings1, embeddings2).cpu().numpy())
        except Exception as e:
            print(f"Error computing cosine similarity for summary: {e}. Returning 0.0 for this metric.")
            cosine_sim_score = 0.0

    summary_accuracy = 1.0 if cosine_sim_score >= 0.25 else 0.0

    security_keywords_in_summary_count = 0
    security_keywords_in_log_count = 0


    return summary_accuracy, rouge1_f1, cosine_sim_score, security_keywords_in_summary_count, security_keywords_in_log_count



def run_mistral_benchmark():
    global sample_outputs_to_print
    global sample_counter

    sample_outputs_to_print = []
    sample_counter = {}

    if model_mistral is None or tokenizer_mistral is None:
        print("Mistral model is not loaded. Cannot run benchmark.")
        return

    results = []

    tasks = ["summary"]
    languages = ["English"]
    instr_types = ["zero-shot"]


    for task_type in tasks:
        for lang in languages:

            if lang == "English":
                prompts_to_iterate = GENERAL_SUMMARIZATION_PROMPTS_EN
            else:
                prompts_to_iterate = {}

            for instr in instr_types:
                for prompt_name, prompt_template in prompts_to_iterate.items():
                    print(f"\n--- Running LogEval for {task_type} ({lang}, {instr}, Prompt: {prompt_name}) ---")

                    dataset_df, labels_dict = _get_dataset(task_type, lang, instr)

                    if dataset_df.empty:
                        print(f"No data found for {task_type}, {lang}, {instr}. Skipping.")
                        continue

                    all_accuracies = []
                    all_f1_scores = []
                    all_inference_times = []
                    all_avg_tokens = []
                    all_cosine_sims = []

                    limited_dataset = dataset_df.head(20)

                    for index, row in limited_dataset.iterrows():
                        log_entry = row["log_entry"]
                        ground_truth_label = labels_dict[row["id"]]


                        actual_instruction_for_model = prompt_template



                        predicted_response, inference_time, generated_tokens_count, security_keywords_in_summary_per_sample, security_keywords_in_log_per_sample = \
                            get_mistral_response(log_entry, actual_instruction_for_model, model_mistral, tokenizer_mistral)

                        current_accuracy = 0.0
                        current_f1_score = 0.0
                        current_cosine_sim = 0.0

                        current_accuracy, current_f1_score, current_cosine_sim, _, _ = \
                            calculate_summary_metrics(predicted_response, ground_truth_label, log_entry) # _ , _ برای ignore کردن کلمات کلیدی

                        all_accuracies.append(current_accuracy)
                        all_f1_scores.append(current_f1_score)
                        all_inference_times.append(inference_time)
                        all_avg_tokens.append(generated_tokens_count)
                        all_cosine_sims.append(current_cosine_sim)

                    avg_accuracy = sum(all_accuracies) / len(all_accuracies) if all_accuracies else 0
                    avg_f1_score = sum(all_f1_scores) / len(all_f1_scores) if all_f1_scores else 0
                    avg_inference_time = sum(all_inference_times) / len(all_inference_times) if all_inference_times else 0
                    avg_avg_tokens = sum(all_avg_tokens) / len(all_avg_tokens) if all_avg_tokens else 0
                    avg_cosine_sim = sum(all_cosine_sims) / len(all_cosine_sims) if all_cosine_sims else 0


                    print(f"\nAverage Accuracy for {task_type} ({lang}, {instr}, Prompt: {prompt_name}): {avg_accuracy:.4f}")
                    print(f"Average F1-score for {task_type} ({lang}, {instr}, Prompt: {prompt_name}): {avg_f1_score:.4f}")
                    print(f"Average Inference Time for {task_type} ({lang}, {instr}, Prompt: {prompt_name}): {avg_inference_time:.4f}s")
                    print(f"Average Generated Tokens for {task_type} ({lang}, {instr}, Prompt: {prompt_name}): {avg_avg_tokens:.2f}")
                    print(f"Average Cosine Similarity for {task_type} ({lang}, {instr}, Prompt: {prompt_name}): {avg_cosine_sim:.4f}")


                    result_entry = {
                        "model": "Mistral-7B-Instruct-v0.2 (LLM)",
                        "task": task_type,
                        "lang": lang,
                        "instr": instr,
                        "prompt_name": prompt_name,
                        "accuracy": avg_accuracy,
                        "f1_score": avg_f1_score,
                        "inference_time": avg_inference_time,
                        "average_tokens": avg_avg_tokens,
                        "cosine_similarity": avg_cosine_sim,

                    }
                    results.append(result_entry)

    results_df = pd.DataFrame(results)
    print("\nBenchmark Results:")
    print(results_df)

    results_df.to_csv("mistral_benchmark_results2.csv", mode='a', header=not os.path.exists("mistral_benchmark_results2.csv"), index=False)
    print("\nResults saved to mistral_benchmark_results2.csv")


if __name__ == "__main__":
    run_mistral_benchmark()

Loading SentenceTransformer model for Cosine Similarity...
SentenceTransformer model loaded.
Logged in to Hugging Face successfully.
Loading tokenizer for mistralai/Mistral-7B-Instruct-v0.2...
Tokenizer loaded.
Loading model mistralai/Mistral-7B-Instruct-v0.2 with 4-bit quantization and CPU offload...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Mistral-7B-Instruct-v0.2 loaded successfully with offload!

--- Running LogEval for summary (English, zero-shot, Prompt: Prompt1_ConciseEvent) ---
Attempting to load data from: dataset/log_summary/log_summary_zero_shot_evaluation_set_english.json
Check existence for this path: True
Successfully loaded 200 samples for summary, English, zero-shot.


/tmp/ipython-input-18-2765742880.py:189: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cosine_sim_score = float(util.cos_sim(embeddings1, embeddings2).cpu().numpy())



Average Accuracy for summary (English, zero-shot, Prompt: Prompt1_ConciseEvent): 1.0000
Average F1-score for summary (English, zero-shot, Prompt: Prompt1_ConciseEvent): 0.0931
Average Inference Time for summary (English, zero-shot, Prompt: Prompt1_ConciseEvent): 9.5414s
Average Generated Tokens for summary (English, zero-shot, Prompt: Prompt1_ConciseEvent): 98.65
Average Cosine Similarity for summary (English, zero-shot, Prompt: Prompt1_ConciseEvent): 0.4541

--- Running LogEval for summary (English, zero-shot, Prompt: Prompt2_SystemAdminRole) ---
Attempting to load data from: dataset/log_summary/log_summary_zero_shot_evaluation_set_english.json
Check existence for this path: True
Successfully loaded 200 samples for summary, English, zero-shot.


...


Average Accuracy for summary (English, zero-shot, Prompt: Prompt2_SystemAdminRole): 0.9000
Average F1-score for summary (English, zero-shot, Prompt: Prompt2_SystemAdminRole): 0.0462
Average Inference Time for summary (English, zero-shot, Prompt: Prompt2_SystemAdminRole): 15.8949s
Average Generated Tokens for summary (English, zero-shot, Prompt: Prompt2_SystemAdminRole): 188.35
Average Cosine Similarity for summary (English, zero-shot, Prompt: Prompt2_SystemAdminRole): 0.4439

--- Running LogEval for summary (English, zero-shot, Prompt: Prompt3_PatternDetection) ---
Attempting to load data from: dataset/log_summary/log_summary_zero_shot_evaluation_set_english.json
Check existence for this path: True
Successfully loaded 200 samples for summary, English, zero-shot.


/tmp/ipython-input-18-2765742880.py:189: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cosine_sim_score = float(util.cos_sim(embeddings1, embeddings2).cpu().numpy())



Average Accuracy for summary (English, zero-shot, Prompt: Prompt3_PatternDetection): 1.0000
Average F1-score for summary (English, zero-shot, Prompt: Prompt3_PatternDetection): 0.0572
Average Inference Time for summary (English, zero-shot, Prompt: Prompt3_PatternDetection): 14.7650s
Average Generated Tokens for summary (English, zero-shot, Prompt: Prompt3_PatternDetection): 174.90
Average Cosine Similarity for summary (English, zero-shot, Prompt: Prompt3_PatternDetection): 0.4849

--- Running LogEval for summary (English, zero-shot, Prompt: Prompt4_ProblemAnalysis) ---
Attempting to load data from: dataset/log_summary/log_summary_zero_shot_evaluation_set_english.json
Check existence for this path: True
Successfully loaded 200 samples for summary, English, zero-shot.


/tmp/ipython-input-18-2765742880.py:189: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cosine_sim_score = float(util.cos_sim(embeddings1, embeddings2).cpu().numpy())



Average Accuracy for summary (English, zero-shot, Prompt: Prompt4_ProblemAnalysis): 1.0000
Average F1-score for summary (English, zero-shot, Prompt: Prompt4_ProblemAnalysis): 0.0461
Average Inference Time for summary (English, zero-shot, Prompt: Prompt4_ProblemAnalysis): 15.7585s
Average Generated Tokens for summary (English, zero-shot, Prompt: Prompt4_ProblemAnalysis): 187.20
Average Cosine Similarity for summary (English, zero-shot, Prompt: Prompt4_ProblemAnalysis): 0.4608

Benchmark Results:
                            model     task     lang      instr  \
0  Mistral-7B-Instruct-v0.2 (LLM)  summary  English  zero-shot   
1  Mistral-7B-Instruct-v0.2 (LLM)  summary  English  zero-shot   
2  Mistral-7B-Instruct-v0.2 (LLM)  summary  English  zero-shot   
3  Mistral-7B-Instruct-v0.2 (LLM)  summary  English  zero-shot   

                prompt_name  accuracy  f1_score  inference_time  \
0      Prompt1_ConciseEvent       1.0  0.093067        9.541407   
1   Prompt2_SystemAdminRole       

In [ ]:
!python run_benchmark.py